In [ ]:
!pip install -r /content/drive/MyDrive/MultiQR_Hackathon/yolov9/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.6 MB/s eta 0:00:00


In [ ]:
!python /content/drive/MyDrive/MultiQR_Hackathon/yolov9/train_dual.py --batch 8 --cfg /content/drive/MyDrive/MultiQR_Hackathon/yolov9/models/detect/custom-yolov9-s.yaml --epochs 100 --data /content/drive/MyDrive/MultiQR_Hackathon/yolov9/data/custom.yaml --hyp /content/drive/MyDrive/MultiQR_Hackathon/yolov9/data/hyps/hyp.scratch-high.yaml --weights "/content/drive/MyDrive/MultiQR_Hackathon/yolov9/yolov9-s.pt" --device 0

2025-10-01 09:19:16.184817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759310356.218086   14974 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759310356.229286   14974 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759310356.254491   14974 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759310356.254528   14974 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759310356.254536   14974 computation_placer.cc:177] computation placer alr

In [42]:
!python /content/drive/MyDrive/MultiQR_Hackathon/yolov9/detect_dual.py --weights /content/drive/MyDrive/MultiQR_Hackathon/yolov9/runs/train/exp/weights/best.pt --source /content/drive/MyDrive/MultiQR_Hackathon/QR_Dataset/test_images/ --save-txt --save-conf

detect_dual: weights=['/content/drive/MyDrive/MultiQR_Hackathon/yolov9/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/MultiQR_Hackathon/QR_Dataset/test_images/, data=drive/MyDrive/MultiQR_Hackathon/yolov9/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/MyDrive/MultiQR_Hackathon/yolov9/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
custom-yolov9-s summary: 658 layers, 9598022 parameters, 0 gradients, 38.7 GFLOPs
image 1/50 /content/drive/MyDrive/MultiQR_Hackathon/QR_Dataset/test_images/img201.jpg: 640x384 1 QR, 72.5ms
image 2/50 /content/drive/MyDrive/MultiQR_Hackathon/QR_D

In [44]:
import glob
from IPython.display import Image, display

i = 0
limit = 1000
for imageName in glob.glob('/content/drive/MyDrive/MultiQR_Hackathon/yolov9/runs/detect/exp_test_images/img208.jpg'):
  if i<limit:
    display(Image(filename=imageName))
    print("\n")
  i = i+1

Output hidden; open in https://colab.research.google.com to view.

Converting annotations to json for test images

In [2]:
import os
import glob
import json
import cv2


pred_folder = '/content/drive/MyDrive/MultiQR_Hackathon/yolov9/runs/detect/exp_test_images/labels'
test_images_folder = '/content/drive/MyDrive/MultiQR_Hackathon/QR_Dataset/test_images'

output_json = '/content/drive/MyDrive/MultiQR_Hackathon/QR_Dataset/submission_detection_1.json'

submission_list = []

for txt_file in sorted(glob.glob(os.path.join(pred_folder, '*.txt'))):
    image_id = os.path.splitext(os.path.basename(txt_file))[0]

    img_path = os.path.join(test_images_folder, image_id + '.jpg')
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    qrs_list = []

    with open(txt_file, 'r') as f:
        lines = f.read().strip().split('\n')
        for line in lines:
            if line == '':
                continue
            parts = line.split()
            if len(parts) < 6:
                continue

            x_center, y_center, width, height, conf = map(float, parts[1:6])

            if conf < 0.56:
                continue

            x_center *= w
            y_center *= h
            width *= w
            height *= h

            x_min = x_center - width/2
            y_min = y_center - height/2
            x_max = x_center + width/2
            y_max = y_center + height/2

            qrs_list.append({"bbox": [x_min, y_min, x_max, y_max]})

    submission_list.append({"image_id": image_id, "qrs": qrs_list})

# Write to JSON
with open(output_json, 'w') as f:
    json.dump(submission_list, f, indent=4)

print(f"Submission JSON saved at: {output_json}")


Submission JSON saved at: /content/drive/MyDrive/MultiQR_Hackathon/QR_Dataset/submission_detection_1.json
